In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
os.environ["KERAS_BACKEND"] = "torch"
import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:

print(X_train.shape)
pd.DataFrame(X_train).head(2)

(11610, 8)


,0,1,2,3,4,5,6,7
0,-0.267667,0.670188,-0.566335,-0.185107,-0.147151,-0.000815,-0.817642,0.610572
1,-1.224056,-0.921890,-0.828655,-0.106856,1.603612,0.382489,0.766641,-0.446348


In [5]:
model = keras.models.Sequential([
keras.layers.Input(shape=X_train.shape[1:]),
keras.layers.Dense(30, activation="relu"),
keras.layers.Dense(1)
])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 30)             │           270 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 301 (1.18 KB)

 Trainable params: 301 (1.18 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
import time
model.compile(loss="mean_squared_error", optimizer="sgd")

tensorboard_cb = keras.callbacks.TensorBoard(f'log/log_{time.strftime("run_%Y_%m_%d-%H_%M_%S")}')

history = model.fit(X_train, y_train, epochs=20,validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 1.2437 - val_loss: 15.7361
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.5459 - val_loss: 0.4709
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4350 - val_loss: 0.4071
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4167 - val_loss: 0.4030
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3954 - val_loss: 0.3817
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3688 - val_loss: 0.3793
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3790 - val_loss: 0.3732
Epoch 8/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3745 - val_loss: 0.3725
Epoch 9/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3616 - val_loss: 0.3706
Epoch 10/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3424 - val_loss: 0.3690
Epoch 11/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3863 - val_loss: 0.3672
Epoch 12/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/ste

In [7]:
model.save("my_keras_model.h5")
model = keras.models.load_model("my_keras_model.h5")

In [8]:
mse_test = model.evaluate(X_test, y_test)
print(mse_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
y_pred

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3575
0.36355045437812805
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[2.6906235],
       [2.0147777],
       [2.1106129]], dtype=float32)